In [16]:
import cv2

img = cv2.imread(filename="image/id.png")
cv2.imshow("ID Card", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import sqlite3

# Connect to the existing database
conn = sqlite3.connect("database/users.db")
cursor = conn.cursor()

# Query all users
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()  # fetch all results

# Print each user's info
for row in rows:
    full_name, id_number, email, nationality, address = row
    print(f"Name: {full_name}")
    print(f"ID: {id_number}")
    print(f"Email: {email}")
    print(f"Nationality: {nationality}")
    print(f"Address: {address}")
    print("-" * 30)

conn.close()


Name: JOHN DOE
ID: JB-007
Email: john@example.com
Nationality: AMERICAN
Address: 123 MAIN STREET, YOUR CITY, USA 789456
------------------------------


In [1]:
import cv2
import pytesseract


pytesseract.pytesseract.tesseract_cmd = 'R:/Program Files/Tesserat-OCR/tesseract.exe'

# ---- Load the image (must be in same folder or give full path) ----
image_path =img = "image/id.png"
  # Replace with your actual image
img = cv2.imread(image_path)

# Optional preprocessing (improves OCR results)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
# You can also use thresholding or blurring to clean noise if needed

# ---- Extract text using OCR ----
text = pytesseract.image_to_string(gray)

# ---- Print extracted text ----
print("---- Extracted Text from ID ----")
print(text)


---- Extracted Text from ID ----
PASS Proof of Age Card CIT: ZENCARD

Name
Angela
Greene

DoB

09 Nov 2002

Expires on

30 Apr 2028
Vo” 5843 2166 1964 2184

=NPCC Sia : +
= National Police Chiefs’ Council Security Industry Authority 1 8




In [23]:
import re

ocr_text = text

# More flexible regex patterns
name_match = re.search(r'[Nn]ame[:\-]?\s*(.+)', ocr_text)
id_match = re.search(r'[Ii][Dd][\s\-]*[Nn]o[:\-]?\s*(\d+)', ocr_text)
dob_match = re.search(r'(DOB|Date of Birth)[:\-]?\s*(\d{2}/\d{2}/\d{4})', ocr_text)

# Extracted Data
extracted_data = {
    "name": name_match.group(1).strip() if name_match else None,
    "id_number": id_match.group(1).strip() if id_match else None,
    "dob": dob_match.group(2).strip() if dob_match else None  # group(2) because we captured two parts
}

print("\nExtracted Data:")
print(extracted_data)


Extracted Data:
{'name': 'Angela', 'id_number': None, 'dob': None}


In [2]:
import easyocr
import cv2
import matplotlib.pyplot as plt


In [9]:


# Load image
image_path = "image/id.png"

# Create OCR reader (English)
reader = easyocr.Reader(['en'])  # Add more languages if needed

# Run OCR
results = reader.readtext(image_path)

# Display results
for (bbox, text, prob) in results:
    print(f"Detected: {text} (Confidence: {prob:.2f})")


Detected: RhRn Anq htoq PP& os-a+P hce (Confidence: 0.11)
Detected: Addis Ababa City Resident ID Card (Confidence: 0.83)
Detected: P9e 770 (Confidence: 0.14)
Detected: No AA0001391718 (Confidence: 0.46)
Detected: Poes.EAD No (Confidence: 0.15)
Detected: IDIADAO-/00876752 (Confidence: 0.35)
Detected: ae4 nge (Confidence: 0.13)
Detected: oUoPE PntA04 (Confidence: 0.05)
Detected: Full Name (Confidence: 0.93)
Detected: MOHAMMED YASIN AWOL (Confidence: 0.92)
Detected: Pto42 + /DOB +U4 15. 1994/DEC 24. 2001 (Confidence: 0.12)
Detected: Pege AEit/Blood Group NA (Confidence: 0.41)
Detected: #Sex (Confidence: 0.22)
Detected: ptnmit # /ssue Dt (Confidence: 0.14)
Detected: J4C 242015/De< 03.2022 (Confidence: 0.20)
Detected: Felfasnt + /Expiry Dt (Confidence: 0.11)
Detected: JRC 24.2019/0-< 03.2028 (Confidence: 0.20)
Detected: /Reg: (Confidence: 0.39)


In [10]:
import re

# Keywords to look for (lowercase for matching)
KEYWORDS = ['dob', 'expires', 'issued', 'id', 'number', 'no']

# Normalize text
lines = [line.strip() for (_, line, _) in results]
lines_lower = [l.lower() for l in lines]

name = None
dob = None
id_number = None

# 1. Find name lines
for idx, line in enumerate(lines_lower):
    if 'name' in line:
        name_lines = []
        for original_line in lines[idx + 1:]:
            l = original_line.lower()
            if any(k in l for k in KEYWORDS):
                break
            if original_line.strip():  # Skip empty lines
                name_lines.append(original_line.strip())
        name = " ".join(name_lines)
        break

# 2. DOB regex: day or month first formats like "09 Nov 2002"
for l in lines:
    match = re.search(r'\b(\d{1,2}[ /-]?[A-Za-z]{3,9}[ /-]?\d{2,4})\b', l)
    if match:
        dob = match.group(1)
        break

# 3. ID number: e.g., "1234 5678 9012 3456" or "1234-5678-9012-3456"
for l in lines:
    match = re.search(r'\b(\d{4}[\s\-]?\d{4}[\s\-]?\d{4}[\s\-]?\d{4})\b', l)
    if match:
        id_number = re.sub(r'[\s\-]', '', match.group(1))
        break

# 🔍 Final Output
extracted_data = {
    "name": name.title() if name else None,
    "dob": dob,
    "id_number": id_number
}

print("🧾 Extracted Data:", extracted_data)


🧾 Extracted Data: {'name': 'Mohammed Yasin Awol', 'dob': None, 'id_number': None}
